# Градиентный бустинг

Мы возвращаемся на титаник, но уже с новым инструментом!

**Большая задача** этой лабораторной - произведите исследование метода градиентного бустинга для классификации [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html), как мы изучали метод случайного леса.

1. Давайте пройдемся по основным моментам:
- начните с разработки baseline модели,
- далее проверьте влияние гиперпараметров на результат работы модели,
- проведите расширенный анализ данных (дополнительная обработка - чистка, новые признаки и т.д.) и оцените работу модели;
- поиском определите лучшую модель;

2. Далее сравниваем работу моделей (леса и бустинга) при их лучших гиперпараметрах по показателям на тестовой выборке (*hold-out*).

> Помним, что поиск лучших гиперпараметров производится кросс-валидацией на обучающей выборке!

3. Напоследок, установите фреймворки [XGBoost](https://xgboost.readthedocs.io/en/latest/) и [LightGBM](https://lightgbm.readthedocs.io/en/latest/), а после этого разберитесь с тем, как создать объекты моделей и с помощью уже известных инструментов найдите лучшие гиперпараметры. 

В результаты должна получиться табличка сравнения, в которой отражены показатели моделей:
- RandomForestClassifier;
- GradientBoostingClassifier;
- XGBoost;
- LightGBM.

Для каждой лучшей модели постройте матрицу ошибок, отобразите результаты и проведите анализ ошибок одной из моделей бустинга (лучшей).


В качестве выводов напишите:
- В чем разница моделей бустинга?
- За что отвечает каждый гиперпараметр моделей бустинга?
- В каких случаях лучше применять один или другой метод бустинга или случайнй лес?
- Какой еще актуальный метод бустинга существует сегодня (от Яндекса)? В чем его преимущества?

(1)импорт данных

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
titanic = pd.read_csv('C:\ml_edu-master\datasets\Titanic_train.csv')

(2)Выбор функции

In [2]:
X = titanic[['Pclass', 'Age', 'Sex']]
y = titanic['Survived']

(3)Очистка данных

In [3]:
X['Age'].fillna(X['Age'].mean(), inplace=True)

(4)Разделите тренировочный набор тестовый набор

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 33)

(5)Функции преобразования

In [5]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False)
X_train = vec.fit_transform(X_train.to_dict(orient='record'))
print(vec.feature_names_)
X_test = vec.transform(X_test.to_dict(orient='record'))

['Age', 'Pclass', 'Sex=female', 'Sex=male']


(6)Моделирование и тестирование производительности

① Единое дерево решений

In [6]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_predict = dtc.predict(X_test)
from sklearn.metrics import classification_report
print(dtc.score(X_test, y_test))
print(classification_report(y_predict, y_test, target_names = ['died', 'survived']))

0.8340807174887892
              precision    recall  f1-score   support

        died       0.90      0.84      0.87       143
    survived       0.74      0.82      0.78        80

    accuracy                           0.83       223
   macro avg       0.82      0.83      0.82       223
weighted avg       0.84      0.83      0.84       223



②Дерево решений градиентного подъема

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()
gbc.fit(X_train,y_train)
gbc_y_prde=gbc.predict(X_test)
print(gbc.score(X_test, y_test))
print(classification_report(gbc_y_prde, y_test, target_names = ['died', 'survived']))

0.8430493273542601
              precision    recall  f1-score   support

        died       0.91      0.84      0.87       145
    survived       0.74      0.85      0.79        78

    accuracy                           0.84       223
   macro avg       0.83      0.84      0.83       223
weighted avg       0.85      0.84      0.85       223



Сравните лес случайных решений и способность модели XGBoost предсказать выживут ли пассажиры на Титанике：

In [8]:
X = titanic[['Pclass', 'Age', 'Sex']]
y = titanic['Survived']
X['Age'].fillna(X['Age'].mean(), inplace=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 33)
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False)
X_train = vec.fit_transform(X_train.to_dict(orient='record'))
X_test = vec.transform(X_test.to_dict(orient='record'))
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
print ( "The accuracy of Random Forest Classifier on testing set:", rfc.score(X_test, y_test))

The accuracy of Random Forest Classifier on testing set: 0.8340807174887892


In [9]:
# Предсказать тот же набор тестов с настроенной по умолчанию моделью XGBoost
from xgboost import XGBClassifier
xgbc = XGBClassifier(learning_rate=0.1)
xgbc.fit(X_train, y_train)
print ('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test, y_test))

[14:07:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The accuracy of eXtreme Gradient Boosting Classifier on testing set: 0.8251121076233184
